In [1]:
import os, time, re, random, glob, json, jieba
import numpy as np
import pandas as pd
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
    default_data_collator,
    TextGenerationPipeline
)

In [2]:
device="cuda:0"
# device="cpu"

In [3]:
CLEAN_TEXT_PATTERN = re.compile(r"[\r\n]")

def clean_text(text):
    return CLEAN_TEXT_PATTERN.sub("", text)

# SFT

In [3]:
import sentencepiece
model_file = "/Users/zeyesun/Documents/Data/models/pangu-350M/vocab.model"
sp = sentencepiece.SentencePieceProcessor()
sp.Load(model_file=model_file)

True

In [4]:
for i in range(10):
    print(sp.id_to_piece(i))

<unk>
<s>
</s>
▃
▂
<sep>
<pad>
<mask>
<eod>
<eot>


In [4]:
# model_name_or_path = "D:\\Data\\models\\pangu_2_6B"
model_name_or_path = "D:\\Data\\models\\pangu-350M"
# model_name_or_path = "/Users/zeyesun/Documents/Data/models/pangu-350M"
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=True)
tokenizer.add_special_tokens({'eos_token': "<eot>", 'pad_token': "<pad>", "sep_token": "<sep>"})

Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.


0

In [5]:
model = AutoModelForCausalLM.from_pretrained(model_name_or_path, trust_remote_code=True, use_cache=False)
model.resize_token_embeddings(len(tokenizer.sp))
model.config.end_token_id = tokenizer.eos_token_id
model.config.pad_token_id = tokenizer.pad_token_id
# model.config.max_length_prompt = 200
model.to(device)
# print(model.device)

Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
Some weights of GPTPanguForCausalLM were not initialized from the model checkpoint at D:\Data\models\pangu-350M and are newly initialized: ['transformer.h.13.attn.masked_bias', 'transformer.h.12.attn.masked_bias', 'transformer.h.2.attn.bias', 'transformer.h.14.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'lm_head.weight', 'transformer.h.8.attn.bias', 'transformer.h.6.attn.bias', 'transformer.h.22.attn.bias', 'transformer.h.15.attn.masked_bias', 'transformer.h.23.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.17.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.18.attn.masked_bias', 'transformer.h.20.attn.masked_bias', 'tran

GPTPanguForCausalLM(
  (transformer): GPTPanguModel(
    (wte): Embedding(40000, 1024)
    (wpe): Embedding(1024, 1024)
    (wqe): Embedding(1024, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPTPanguBlock(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPTPanguAttention(
          (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (c_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): GPTPanguMLP(
          (c_fc): Linear(in_features=1024, out_features=4096, bias=True)
          (c_proj): Linear(in_features=4096, out_features

### Data Processing

In [6]:
# f = "/Users/zeyesun/Documents/Data/raw/baike_qa2019/baike_qa_train.json"
f = "D:\\Data\\raw\\baike_qa_train.json"
items = []
lens_prompt = []
lens_label = []
with open(f, "r", encoding="utf-8") as r:
    while True:
        line = r.readline()
        if not line:
            break
        item = json.loads(line.strip("\n"))
        prompt = clean_text(item['title'] if len(item['title']) > len(item['desc']) else item['desc'])
        label = clean_text(item['answer'])
        items.append(item)
        lens_prompt.append(len(prompt))
        lens_label.append(len(label))
print(len(items))
print(np.percentile(lens_prompt, np.arange(90, 101)))
print(np.percentile(lens_label, np.arange(90, 101)))

1425170


In [43]:
i = 789
items[i]

{'qid': 'qid_7770627748113178417',
 'category': '生活-保健养生',
 'title': '减肥健身我要下载一套很好的有氧健身操，请问哪里有下载地址，谢谢！ ',
 'desc': '我要下载一套很好的有氧操，请问哪里有下载地址，谢谢！',
 'answer': '我看练太极拳就可以，该运动也是有氧健身的运动，长期锻炼还可以治疗很多慢性病。'}

In [44]:
items[i]['answer']

'我看练太极拳就可以，该运动也是有氧健身的运动，长期锻炼还可以治疗很多慢性病。'

### prediction

In [6]:
# checkpoint = "/Users/zeyesun/Documents/Data/output/sft/pangu-350M/checkpoint-12000/pytorch_model.bin"
checkpoint = "D:\\Data\\output\\sft\\pangu-350M\\checkpoint-12000\\pytorch_model.bin"
st = torch.load(checkpoint, map_location="cpu")
model.load_state_dict(st)

<All keys matched successfully>

In [7]:
text_generator = TextGenerationPipeline(model, tokenizer, device=device)

The model 'GPTPanguForCausalLM' is not supported for . Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegatronBertForCausalLM', 'MvpForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLMHead', 'RemBertForCausalLM', 'RobertaForCausalLM', 'RobertaPreLayerNormForCausalLM', 'RoCBertForCausalLM', 'RoFormerForCausalLM', 'Speech2Text2ForCausalLM', 'TransfoXLLMHeadModel', 'TrOCRForCau

In [8]:
# f = "/Users/zeyesun/Documents/Data/raw/baike_qa2019/baike_qa_train.json"
f = "D:\\Data\\raw\\baike_qa_train.json"
i = 0
prompts = []
prompts_processed = []
labels = []
with open(f, "r", encoding="utf-8") as r:
    while True:
        line = r.readline()
        if not line:
            break
        item = json.loads(line.strip("\n"))
        prompt = clean_text(item['title'] if len(item['title']) > len(item['desc']) else item['desc'])
        label = clean_text(item['answer'])
        prompt_processed = prompt + tokenizer.sep_token + "模型回答:"
        prompts.append(prompt)
        prompts_processed.append(prompt_processed)
        labels.append(label)
        i += 1
        if i > 1000:
            break

In [14]:
num_return_sequences = 5
i = 10
j = 20
t1 = time.time()
results = text_generator(prompts_processed[i:j], max_length=200, num_return_sequences=num_return_sequences,
                         do_sample=True, top_k=50, temperature=10.0)
print(f"Finished prediction, time taken: {time.time()-t1}")

for prompt, res, label in zip(prompts[i:j], results[:(j-i)], labels[i:j]):
    print(f"prompt: {prompt}\nlabel: {label}")
    for k in range(num_return_sequences):
        model_answer = res[k]['generated_text'].split("模型回答:")[1].replace("<eot>", "").replace("<pad>", "")
        print(f"model answer-{k}: {model_answer}")
    print("\n\n")

prompt: 有6区无尽之海众神之子工会的吗?你们会怎么了?看不见人了,解散了 
label: 我听原众神成员说 众神的副会长 卷了工会财产跑路了(几千W呢)  所以导致工会解散 哎 真是人心陷恶   
model answer-0: 我也是无尽之海的,现在工会还没解散,是不可能解散了,不过我想你应该有很多个工会吧,应该有很多人在等着你,如果你有足够的耐心的话,可以去工会看看有多少人在等着你,我有个朋友就在等着我,我说的是在无尽之海,你知道的,要是没人的话,你就去工会看看,反正现在工会也没解散,你可以去公会看看,我也是在无尽之海的,在无尽之海也有很多人在等着你,要是你能坚持的话,我想你会更好的!结舌
model answer-1: 现在工会也解散了,是吧··タタタ瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄瞄
model answer-2: 你可以去看看万盛万盛万盛万盛万盛万盛万盛万盛万盛万盛万盛万盛万盛万盛万盛万盛万盛NdF万盛NdF
model answer-3: 不解散了,我是6区无尽之海的,我想知道,如果解散的话,工会的人和工会的人一起走,会怎么样?方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒方寒
model answer-4: 有,因为工会的人太多了,服务器都快封了。タ榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫榫



prompt: 老师好，

In [20]:
prompt = "内有吊车，牛腿高度怎么定？根据什么来定？生产的工艺要求？还是要考虑到其他因素？"
prompt_processed = prompt + tokenizer.sep_token + "模型回答:"
num_return_sequences=5
res = text_generator(prompt_processed, max_length=200, num_return_sequences=num_return_sequences,
                         do_sample=True, top_p=0.5, temperature=10.0)
print(f"prompt: {prompt}")
for i in range(num_return_sequences):
    model_answer = res[i]['generated_text'].split("模型回答:")[1].replace("<eot>", "").replace("<pad>", "")
    # print(res)
    print(f"model answer-{i}: {model_answer}")

prompt: 内有吊车，牛腿高度怎么定？根据什么来定？生产的工艺要求？还是要考虑到其他因素？
model answer-0: 应该没有规定棍子高度标准.可以按以下几点.⁇底高度一般是在5米~10,1M;长度要长1-5;宽度是要=50cmx2‰;用到2支牛手:50、±@+50,<E8%]50:4,0x20^40*80;30、100、200*60^60;400X200、60X60x400(按2/2.5或2.5/4折算成4/6则需要5米长牛棒,长4倍多就行了>>长度、用@:4-x0-60,X50x50*40.根据不同牛体选用量。用4-50-
model answer-1: 用“体型和质量尺寸比例关系方准的尺寸精度”计算出所规定范围和长度尺寸;在此中应确定每段高度是根据“零件加工质量几何图形表示、轮廓线”而进行绘制的:体型即加工质量(T1)图2某产品零件加工后工序顺序图 工位表:工号:生产周期/月或工作量Tmax工作频/H/每根线每道每排加工的总小时工时数Sum:(长度或尺寸的量化位数(如“1#-10/4);3’+10%+1]≥40°+25m/3’√H2,2,1+40%’为(长,(
model answer-2: 生产用什么,比如汽车发动机还是什么整机什么,在用也罢(包括生产配件也就是模具〕要考虑什么等等要在图纸中有交代你!具体你在厂牌里应该了解具体一点的'具体就多些的啊~)希望对题!希望会有助!你就在这里留言你也看!如果帮到点了记得留下宝贵的经验!谢字在我评论上.还有一点你如果想在淘宝里开卖点精致或者你不嫌脏,还是去买个牌子货.不就知道买哪个比较的?其实也不要急着做大了 如果有问题也会有个问题找商家给你答复哦如果有的没的的买还是建议直接买,不急是吗
model answer-3: 一般说来长度应该和牛肩高和长度对应成1.15M或者1.26MM为适合吊钩'子牛吊杆高度为1.8,2m的高度.你要买什么样大小才方便,可以买5'8MM或者10～15的规格尺寸吊在车钩梢的一头和钩的两边即可根据你是哪种规格你吊多少高度的产品一般要考虑下面三块问题.首先:如果吊机要保证在任何的作业场所都可顺利操作(如在高处起料时就应该是高度超过牛的高度3米才对)其次如果要对货要保持足够空间的高度最后根据市场情况确定货的多少你选择挂的时候注意下挂钩是对勾 如果不
model answer-4: 要具体

# Reward Model

In [9]:
model_name_or_path = "D:\\Data\\models\\pangu-350M"
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_cache=False, trust_remote_code=True)
tokenizer.add_special_tokens({'unk_token': "<unk>",
                                  'bos_token': "<s>",
                                  'eos_token': "<eot>",
                                  'pad_token': "<pad>",
                                  "sep_token": "<sep>"})

Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


0

In [10]:
max_length = 1024
text = "你好，你是谁"
# text = "<|startoftext|>" + text + "<|endoftext|>"
res = tokenizer(text, max_length=max_length, truncation="longest_first", 
          return_tensors="pt", add_special_tokens=False)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\SUNZEY~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.501 seconds.
Prefix dict has been built successfully.


In [19]:
res.keys()
# torch.cat((res['input_ids'], res['input_ids']), axis=1)

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])